In [1]:
import pandas as pd
import numpy as np
import pickle

#bbcのニュース記事のデータを用いて、英文ニュースのジャンルを
#５つのジャンルで分類するモデルをkerasを焼死作成

In [2]:
import glob
import os
os.chdir(r"C:\Users\81702\kagglechallnge\bbc\bbc-fulltext (document classification)\bbc\business")
pathlist = [r"C:\Users\81702\kagglechallnge\bbc\bbc-fulltext (document classification)\bbc\business", 
            r"C:\Users\81702\kagglechallnge\bbc\bbc-fulltext (document classification)\bbc\entertainment",
           r"C:\Users\81702\kagglechallnge\bbc\bbc-fulltext (document classification)\bbc\politics",
            r"C:\Users\81702\kagglechallnge\bbc\bbc-fulltext (document classification)\bbc\sport",
           r"C:\Users\81702\kagglechallnge\bbc\bbc-fulltext (document classification)\bbc\tech"]
topics = ["business", "entertainment", "politics", "sport", "tech"]


dirPath = r"C:\Users\81702\kagglechallnge\bbc\bbc-fulltext (document classification)\bbc\business\*"
result = glob.glob(dirPath)



In [3]:
allfiles = []
for path in pathlist:
    path = path + "\*"
    files = glob.glob(path)
    allfiles.append(files)
    
print(len(allfiles))

5


In [4]:
alltexts = []
for files, topic in zip(allfiles, topics):
    texts = []
    for file in files:
        f = open(file)
        data = f.read()
        data = [data, topic]
        texts.append(data)
    alltexts.append(texts)
        
print(len(alltexts))

5


In [5]:
print(topics)

['business', 'entertainment', 'politics', 'sport', 'tech']


In [6]:
business = pd.DataFrame(alltexts[0])
entertainment = pd.DataFrame(alltexts[1])
politics = pd.DataFrame(alltexts[2])
sport = pd.DataFrame(alltexts[3])
tech = pd.DataFrame(alltexts[4])

In [7]:
df = pd.DataFrame(alltexts[0])

for i in range(1,4):
    new = pd.DataFrame(alltexts[i])
    df = pd.concat([df, new])

In [8]:
df.columns = ["text", "topic"]
print(df)

                                                  text     topic
0    Ad sales boost Time Warner profit\n\nQuarterly...  business
1    Dollar gains on Greenspan speech\n\nThe dollar...  business
2    Yukos unit buyer faces loan claim\n\nThe owner...  business
3    High fuel prices hit BA's profits\n\nBritish A...  business
4    Pernod takeover talk lifts Domecq\n\nShares in...  business
..                                                 ...       ...
506  Big guns ease through in San Jose\n\nTop-seede...     sport
507  Almagro continues Spanish surge\n\nUnseeded Ni...     sport
508  Melzer shocks Agassi in San Jose\n\nSecond see...     sport
509  Mirza makes Indian tennis history\n\nTeenager ...     sport
510  Roddick to face Saulnier in final\n\nAndy Rodd...     sport

[1824 rows x 2 columns]


In [9]:
train = df["text"]
test = df["topic"]

In [10]:
#テキストの前処理
#改行、大文字小文字の区別、ストップワードに関して
from nltk.corpus import stopwords
stopwords = set(stopwords.words("english"))


def stopword(text):
    new =  " ".join([word for word in str(text).split() if word not in stopwords])
    return new
    
train = train.str.lower()
train = train.str.replace('[^\w\s]','')
train = train.str.replace("\n", "")
train.apply(stopword)
train = train.to_numpy()

In [11]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(train, test, test_size=0.25, shuffle=True, random_state=42)


In [12]:
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer

In [13]:
num_labels = 5
vocab_size = 15000
batch_size = 100

In [14]:
#文章のベクトル化
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(train)

trainx = tokenizer.texts_to_matrix(train_x, mode="tfidf")
testx = tokenizer.texts_to_matrix(test_x, mode="tfidf")

In [15]:
print(trainx.shape)

(1368, 15000)


In [16]:
#カテゴリ変数をエンコード
encoder = LabelBinarizer()
encoder.fit(topics)
train_y = encoder.transform(train_y)
test_y = encoder.transform(test_y)

In [17]:
trainx = trainx.reshape(-1, vocab_size)

In [18]:
model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
history = model.fit(trainx, train_y,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               7680512   
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 2

In [20]:
score = model.evaluate(testx, test_y,
                       batch_size=batch_size, verbose=1)
 
#depp_learningモデルを使用した結果、97％の精度print('Test accuracy:', score[1])
 


5/5 [==============================] - 0s 29ms/step - loss: 0.0739 - accuracy: 0.9759
Test accuracy: 0.9758771657943726


In [22]:
model.model.save('my_model.h5')
 
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


AttributeError: 'Sequential' object has no attribute 'model'